In [1]:
import sqlite3
# Connect to database
db = sqlite3.connect('my_database.db')

## CREATE

In [2]:
# Get a cursor:
cursor = db.cursor()
# Execute a "create table" command:
cursor.execute('''
    CREATE TABLE user(
        id     INTEGER   PRIMARY KEY, 
        name   TEXT,
        phone  TEXT, 
        profession TEXT,
        email  TEXT      UNIQUE, 
        password TEXT)
''')

## INSERT

In [3]:
name1 = 'Tom Pythonovitz'
phone1 = '3366858'
profession1 = "Programmer"
email1 = 'Tom.Pythonovitz@example.com'
# A very secure password
password1 = '12345'

name2 = 'Tammi Pythonovitz'
phone2 = '5557241'
profession2 = "Programmer"
email2 = 'Tammi@example.com'
password2 = 'Javansky'

name3 = 'George Rustinovsky'
phone3 = '33333'
profession3 = "Manager"
email3 = 'GRust@example.com'
password3 = 'Rust for ever'

In [4]:
# Insert user 1 (why question marks? see https://xkcd.com/327/ )
cursor.execute('''
    INSERT INTO user(name, phone, profession, email, password)
    VALUES(?,?,?,?,?)
    ''', (name1, phone1, profession1, email1, password1))
print('First user inserted')
# NOTE: The id is set automatically (incrementally)

# Insert user 2
cursor.execute('''
    INSERT INTO user(name, phone, profession, email, password)
    VALUES(?,?,?,?,?)
    ''', (name2,phone2, profession2, email2, password2))
print('Second user inserted')
# db.rollback()
db.commit()

First user inserted
Second user inserted


In [5]:
# Insert user 3
cursor.execute('''
    INSERT INTO user(name, phone, profession, email, password)
    VALUES(:name, :phone, :profession, :email, :password)''',
    {'name':name3, 'phone':phone3, 'profession':profession3, 'email':email3, 'password':password3})
print('Third user inserted')
db.commit()

Third user inserted


In [6]:
users = [('Joee Javany', '2222', 'Manager', 'joo@example.mail','password'),
        ('Shirel Cplustik', '3333', 'Secretary', 'cpp@example.mail','cppass'),
        ('Adam Kotlinberg', '4444', 'Secretary', 'Adam_Kotlin@example.mail','JustPassword')
        ]
# Insert many
cursor.executemany('''
    INSERT INTO user(name, phone, profession, email, password)
    VALUES(?,?,?,?,?)
''', users)
print('Many users inserted')
db.commit()

Many users inserted


## SELECT

In [7]:
# Selecting all columns (*)
cursor.execute('''
    SELECT * FROM user
''')
for row in cursor:
    print(row)

(1, 'Tom Pythonovitz', '3366858', 'Programmer', 'Tom.Pythonovitz@example.com', '12345')
(2, 'Tammi Pythonovitz', '5557241', 'Programmer', 'Tammi@example.com', 'Javansky')
(3, 'George Rustinovsky', '33333', 'Manager', 'GRust@example.com', 'Rust for ever')
(4, 'Joee Javany', '2222', 'Manager', 'joo@example.mail', 'password')
(5, 'Shirel Cplustik', '3333', 'Secretary', 'cpp@example.mail', 'cppass')
(6, 'Adam Kotlinberg', '4444', 'Secretary', 'Adam_Kotlin@example.mail', 'JustPassword')


In [8]:
# Selecting some columns:
cursor.execute('''
    SELECT name, password 
    FROM user
''')
for row in cursor:
    print(f'{row[0]} : {row[1]}')

Tom Pythonovitz : 12345
Tammi Pythonovitz : Javansky
George Rustinovsky : Rust for ever
Joee Javany : password
Shirel Cplustik : cppass
Adam Kotlinberg : JustPassword


In [11]:
# Query with parameters:
cursor.execute('''
    SELECT name, email, phone 
    FROM user 
    WHERE profession="Programmer"
''')
print(cursor.fetchall())

[('Tom Pythonovitz', 'Tom.Pythonovitz@example.com', '3366858'), ('Tammi Pythonovitz', 'Tammi@example.com', '5557241')]


## UPDATE, DELETE

In [12]:
# Update the phone of user with id 1:
userid = 1
newphone = '4477969'
cursor.execute('''
    UPDATE user 
    SET phone = ? 
    WHERE id = ?
''', (newphone, userid))

# Delete user with id 3:
delete_userid = 3
cursor.execute('''
    DELETE FROM user 
    WHERE id = ?
''', (delete_userid,))

db.commit()

In [13]:
cursor.execute('''
    SELECT id, name,phone  FROM user
''')
for row in cursor:
    print(f'{row[0]} ) {row[1]} : {row[2]}')

1 ) Tom Pythonovitz : 4477969
2 ) Tammi Pythonovitz : 5557241
4 ) Joee Javany : 2222
5 ) Shirel Cplustik : 3333
6 ) Adam Kotlinberg : 4444


## ROLLBACK

In [15]:
cursor.execute('''
    UPDATE user 
    SET phone = ? 
    WHERE id = ? 
''', ("121212", userid))

# The user's phone is not updated
db.rollback()

cursor.execute('''SELECT id, name,phone  FROM user''')
for row in cursor:
    print(f'{row[0]} ) {row[1]} : {row[2]}')

1 ) Tom Pythonovitz : 4477969
2 ) Tammi Pythonovitz : 5557241
4 ) Joee Javany : 2222
5 ) Shirel Cplustik : 3333
6 ) Adam Kotlinberg : 4444


## JOIN

In [16]:
cursor.execute('''
    DROP TABLE IF EXISTS user_pairs
''')
cursor.execute('''
    CREATE  TABLE user_pairs(
        id1    INTEGER,
        id2    INTEGER
    )
''')

cursor.executemany('''
    INSERT INTO user_pairs(id1, id2)
    VALUES(?,?)
''', [(1,2), (5,6)])

cursor.execute('''SELECT * FROM user_pairs''')
for row in cursor: print(row)

(1, 2)
(5, 6)


In [17]:

cursor.execute('''
    SELECT user.name, user_pairs.id1, user_pairs.id2
    FROM user JOIN user_pairs ON(user.id==user_pairs.id1)
''')
for row in cursor: print(row)

('Tom Pythonovitz', 1, 2)
('Shirel Cplustik', 5, 6)


In [18]:

cursor.execute('''
    SELECT user.name, user_pairs.id1, user_pairs.id2
    FROM user JOIN user_pairs ON(user.id==user_pairs.id2)
''')
for row in cursor: print(row)

('Tammi Pythonovitz', 1, 2)
('Adam Kotlinberg', 5, 6)


In [19]:

cursor.execute('''
    SELECT user1.name, user_pairs.id1, user2.name, user_pairs.id2
    FROM user AS user1
    JOIN user_pairs    ON(user1.id==user_pairs.id1)
    JOIN user AS user2 ON(user2.id==user_pairs.id2)
''')
for row in cursor: print(row)

('Tom Pythonovitz', 1, 'Tammi Pythonovitz', 2)
('Shirel Cplustik', 5, 'Adam Kotlinberg', 6)


## GROUP BY

In [20]:
cursor.execute('''
    SELECT profession, COUNT(*)
    FROM user 
    GROUP BY profession
''')
for row in cursor: print(row)

('Manager', 1)
('Programmer', 2)
('Secretary', 2)


In [21]:
cursor.execute('''
    SELECT profession, AVG(ID)
    FROM user 
    GROUP BY profession
''')
for row in cursor: print(row)

('Manager', 4.0)
('Programmer', 1.5)
('Secretary', 5.5)


## close

In [22]:
db.close()

## Converting Python types to SQL types:

•	None type is converted to NULL<br>
•	int type is converted to INTEGER<br>
•	float type is converted to REAL<br>
•	str type is converted to TEXT<br>
•	bytes type is converted to BLOB
